In [1]:
# ##############################################################################
#
# 통합 펩타이드 발굴 파이프라인 (PRODIGY 개선 버전)
# - 라이브러리 설치 중복 방지
# - 파일 구조 개선 (fasta/, pdb/ 폴더 분리)
#
# ##############################################################################

import time
import os
import sys
import site
import subprocess
import shutil

# 파이프라인 시작 시간 기록
pipeline_start_time = time.time()

# ==============================================================================
# STEP 0: 통합 환경 설정 및 라이브러리 설치 (중복 방지)
# ==============================================================================

def check_and_install_libraries():
    """라이브러리 설치 상태를 확인하고 필요한 것만 설치"""
    print("="*80)
    print("STEP 0: 환경 설정 및 라이브러리 설치 (중복 방지)")
    print("="*80)

    # 설치 플래그 파일
    install_flag_file = "peptide_pipeline_installed.flag"

    if os.path.exists(install_flag_file):
        print("✅ 이미 설치된 환경이 감지되었습니다. 설치를 건너뜁니다.")
        return True

    print("🔧 첫 실행입니다. 필요한 라이브러리들을 설치합니다...")

    try:
        # 1. 기본 라이브러리 설치
        print("\n[1/6] 기본 라이브러리 설치...")
        os.system("pip install -q pytz requests beautifulsoup4 openpyxl")

        # 2. ColabFold 설치
        print("\n[2/6] ColabFold 설치...")
        os.system("pip uninstall -y tensorflow tensorboard tb-nightly tensorflow-estimator tensorflow-hub tensorflow-io > /dev/null 2>&1")
        os.system("pip install -q --no-warn-conflicts 'colabfold[alphafold] @ git+https://github.com/sokrypton/ColabFold'")
        os.system("pip install -q --no-warn-conflicts 'jax[cuda11_pip]' -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html")

        # ColabFold 패치
        try:
            dist_packages_path = site.getsitepackages()[0]
            batch_py_path = os.path.join(dist_packages_path, 'colabfold', 'batch.py')
            if os.path.exists(batch_py_path):
                os.system(f"sed -i 's/tf.get_logger().setLevel(logging.ERROR)/#tf.get_logger().setLevel(logging.ERROR)/g' {batch_py_path}")
                print("   > ColabFold 패치 완료")
        except:
            print("   > ColabFold 패치 건너뜀")

        # 3. 펩타이드 생성 모델
        print("\n[3/6] Transformers 설치...")
        os.system("pip install -q --upgrade transformers sentencepiece")

        # 4. 화학 정보학 도구
        print("\n[4/6] 화학 정보학 도구 설치...")
        os.system("apt-get update -qq > /dev/null 2>&1")
        os.system("apt-get install -y --quiet openbabel python3-openbabel libopenbabel-dev > /dev/null 2>&1")
        os.system("pip install -q openbabel-wheel rdkit-pypi biopython ProLIF MDAnalysis oddt scikit-learn plip")

        # 5. AutoDock Vina
        print("\n[5/6] AutoDock Vina 설치...")
        setup_vina_robust()

        # 6. 추가 도구
        print("\n[6/6] 추가 도구 설치...")
        os.system("pip install -q pymol-open-source meeko > /dev/null 2>&1")

        # 설치 완료 플래그 생성
        with open(install_flag_file, 'w') as f:
            f.write(f"Installed at: {time.strftime('%Y-%m-%d %H:%M:%S')}\n")

        print("\n✅ 모든 라이브러리 설치 완료!")
        return True

    except Exception as e:
        print(f"❌ 설치 중 오류 발생: {e}")
        return False

def setup_vina_robust():
    """AutoDock Vina 설치"""
    vina_dir = "vina_1.2.3_linux_x86_64"

    if not os.path.exists(vina_dir):
        download_commands = [
            "wget -q https://github.com/ccsb-scripps/AutoDock-Vina/releases/download/v1.2.3/vina_1.2.3_linux_x86_64.zip",
            "curl -L -o vina_1.2.3_linux_x86_64.zip https://github.com/ccsb-scripps/AutoDock-Vina/releases/download/v1.2.3/vina_1.2.3_linux_x86_64.zip"
        ]

        for cmd in download_commands:
            if os.system(cmd) == 0:
                break

        os.system("unzip -q -o vina_1.2.3_linux_x86_64.zip")

        for executable in [f"{vina_dir}/vina", f"{vina_dir}/bin/vina"]:
            if os.path.exists(executable):
                os.chmod(executable, 0o755)

    # Vina 실행파일 경로 반환
    for path in [f"./{vina_dir}/vina", f"./{vina_dir}/bin/vina", "vina"]:
        if os.path.exists(path) and os.access(path, os.X_OK):
            return os.path.abspath(path)
    return None

# 라이브러리 설치 실행
if not check_and_install_libraries():
    print("❌ 설치 실패. 파이프라인을 종료합니다.")
    sys.exit(1)

# Vina 실행파일 경로 설정
VINA_EXECUTABLE = setup_vina_robust()
PLIP_AVAILABLE = (shutil.which("plipcmd") is not None)

print("\n# 설치된 도구들의 상태 요약")
print(f"   • AutoDock Vina: {'✅ 설치됨' if VINA_EXECUTABLE else '⚠️ 간단한 추정 사용'}")
print(f"   • PLIP: {'✅ 사용 가능' if PLIP_AVAILABLE else '⚠️ 대체 계산 사용'}")
print(f"   • PRODIGY(Web): 🌐 웹 시도/실패 시 대체 점수")
print("="*80)


# ==============================================================================
# STEP 1: 파이프라인 실행 변수 설정 및 폴더 구조 생성
# ==============================================================================

print("\nSTEP 1: 변수 설정 및 폴더 구조 생성")
print("="*80)

import torch
from datetime import datetime
import pytz

# --- 사용자 설정 ---
N_PEPTIDES = 10
TARGET_PROTEIN_SEQUENCE = "AFTVTVPKDLYVVEYGSNMTIECKFPVEKQLDLAALIVYWEMEDKNIIQFVHGEEDLKVQHSSYRQRARLLKDQLSLGNAALQITDVKLQDAGVYRCMISYGGADYKRITVKVNAPYNKINQRILVVDPVTSEHELTCQAEGYPKAEVIWTSSDHQVLSGKTTTTNSKREEKLFNVTSTLRINTTTNEIFYCTFRRLDPEENHTAELVIPELPLAHPPNERT"
PEPTIDE_LENGTH = 4
BASE_FOLDER_PREFIX = "PDP"

# 한국 시간 기반 폴더명 생성
kst = pytz.timezone('Asia/Seoul')
now_kst = datetime.now(kst)
timestamp = now_kst.strftime("%Y%m%d_%H%M%S")
JOB_NAME = f"{BASE_FOLDER_PREFIX}_{timestamp}"

# 📁 개선된 폴더 구조 생성
def create_project_structure():
    """프로젝트 폴더 구조 생성"""
    folders = [
        JOB_NAME,
        os.path.join(JOB_NAME, "fasta"),      # FASTA 파일 전용
        os.path.join(JOB_NAME, "pdb"),        # PDB 파일 전용
        os.path.join(JOB_NAME, "results"),    # 결과 파일
        os.path.join(JOB_NAME, "temp")        # 임시 파일
    ]

    for folder in folders:
        os.makedirs(folder, exist_ok=True)

    print(f"✅ 프로젝트 폴더 구조 생성 완료: {JOB_NAME}")
    return folders

create_project_structure()

# 파일 경로 설정 (개선된 구조)
PROTEIN_FASTA_PATH = os.path.join(JOB_NAME, "fasta", "target_protein.fasta")
OUTPUT_FINAL_XLSX_PATH = os.path.join(JOB_NAME, "results", f"final_ranking_{timestamp}.xlsx")

# 타겟 단백질 FASTA 파일 저장
with open(PROTEIN_FASTA_PATH, "w") as f:
    f.write(f">target_protein\n{TARGET_PROTEIN_SEQUENCE}\n")

print(f"📊 설정 요약:")
print(f"   • 작업 폴더: {JOB_NAME}")
print(f"   • 펩타이드 개수: {N_PEPTIDES}")
print(f"   • 펩타이드 길이: {PEPTIDE_LENGTH}")
print(f"   • 타겟 서열 길이: {len(TARGET_PROTEIN_SEQUENCE)}")
print("="*80)

# ==============================================================================
# STEP 2: ESM-2 펩타이드 생성
# ==============================================================================

print("\nSTEP 2: ESM-2 펩타이드 생성")
print("="*80)

import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForMaskedLM

# ESM-2 모델 로드
model_name = "facebook/esm2_t12_35M_UR50D"
print(f"ESM-2 모델 로딩: {model_name}")
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForMaskedLM.from_pretrained(model_name).to("cuda" if torch.cuda.is_available() else "cpu")

# 펩타이드 생성
formatted_target = " ".join(list(TARGET_PROTEIN_SEQUENCE))
mask_tokens = " ".join([tokenizer.mask_token] * PEPTIDE_LENGTH)
prompt = f"{tokenizer.cls_token} {formatted_target} {tokenizer.eos_token} {mask_tokens}"
input_ids = tokenizer(prompt, return_tensors="pt").input_ids
mask_token_indices = (input_ids == tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0]

peptides = []
peptide_fasta_paths = []

print("\n🧬 펩타이드 생성 중...")
with torch.no_grad():
    for i in range(N_PEPTIDES):
        current_ids = input_ids.clone().to(model.device)
        shuffled_mask_indices = mask_token_indices[torch.randperm(len(mask_token_indices))]

        for mask_idx in shuffled_mask_indices:
            outputs = model(input_ids=current_ids)
            logits = outputs.logits[0, mask_idx, :] / 1.0  # temperature
            top_k_values, top_k_indices = torch.topk(logits, min(50, tokenizer.vocab_size))
            filter_tensor = torch.full_like(logits, -float('Inf'))
            filter_tensor.scatter_(0, top_k_indices, top_k_values)
            probs = F.softmax(filter_tensor, dim=-1)
            predicted_token_id = torch.multinomial(probs, num_samples=1)
            current_ids[0, mask_idx] = predicted_token_id.item()

        generated_token_ids = current_ids[0, mask_token_indices]
        sequence = "".join(tokenizer.decode(generated_token_ids, skip_special_tokens=True).split())
        peptides.append(sequence)

        # 📁 FASTA 폴더에 저장
        fasta_path = os.path.join(JOB_NAME, "fasta", f"peptide_{i}.fasta")
        with open(fasta_path, "w") as f:
            f.write(f">peptide_{i}\n{sequence}\n")
        peptide_fasta_paths.append(fasta_path)
        print(f"  [{i+1:2d}/{N_PEPTIDES}] {sequence}")

print(f"\n✅ {N_PEPTIDES}개 펩타이드 생성 완료")
print("="*80)

# ==============================================================================
# STEP 3: ColabFold 구조 예측
# ==============================================================================

import glob

print("\nSTEP 3: ColabFold 3D 구조 예측")
print("="*80)

# 배치 CSV 파일 생성
batch_csv_path = os.path.join(JOB_NAME, "temp", "batch_complexes.csv")
with open(batch_csv_path, "w") as f:
    f.write("id,sequence\n")
    for i, peptide_seq in enumerate(peptides):
        complex_sequence = f"{TARGET_PROTEIN_SEQUENCE}:{peptide_seq}"
        f.write(f"complex_{i},{complex_sequence}\n")

# ColabFold 실행
output_dir = os.path.join(JOB_NAME, "pdb")  # 📁 PDB 폴더에 직접 저장
log_file = os.path.join(JOB_NAME, "temp", "colabfold.log")

print(f"ColabFold 실행 중... (예상 시간: 10-30분)")
colabfold_cmd = (f"colabfold_batch "
                f"--num-recycle 1 "
                f"--model-type alphafold2_multimer_v3 "
                f"--rank ptm "
                f"--max-msa 32:128 "
                f"--num-models 1 "
                f"--stop-at-score 0.5 "
                f"{batch_csv_path} {output_dir} > {log_file} 2>&1")

result = os.system(colabfold_cmd)

# PDB 파일 수집
predicted_pdb_files = []
for i in range(N_PEPTIDES):
    pdb_pattern = os.path.join(output_dir, f"complex_{i}_unrelaxed_rank_001*.pdb")
    pdb_files = sorted(glob.glob(pdb_pattern))
    if pdb_files:
        predicted_pdb_files.append(pdb_files[0])
        print(f"  ✅ 복합체 {i}: {os.path.basename(pdb_files[0])}")

print(f"\n✅ {len(predicted_pdb_files)}개 구조 예측 완료")
print("="*80)

# ==============================================================================
# STEP 4: pTM 점수 추출
# ==============================================================================

import json
import re

print("\nSTEP 4: pTM 점수 추출")
print("="*80)

ptm_scores_map = {}
score_file_patterns = [
    os.path.join(output_dir, "*_scores.json"),
    os.path.join(output_dir, "complex_*_scores.json")
]

all_score_files = []
for pattern in score_file_patterns:
    all_score_files.extend(glob.glob(pattern))

for score_file in set(all_score_files):
    try:
        match = re.search(r'complex_(\d+)', os.path.basename(score_file))
        if match:
            peptide_index = int(match.group(1))
            with open(score_file, 'r') as f:
                data = json.load(f)
            ptm_score = data.get('ptm', data.get('iptm', 0.0))
            if isinstance(ptm_score, list):
                ptm_score = ptm_score[0] if ptm_score else 0.0

            if peptide_index < len(peptides):
                ptm_scores_map[peptides[peptide_index]] = round(float(ptm_score), 3)
    except:
        continue

print(f"✅ {len(ptm_scores_map)}개 pTM 점수 추출 완료")
print("="*80)

# ==============================================================================
# STEP 5: PRODIGY(웹) 및 상호작용 분석 함수 (PLIP 우선 → 실패 시 대체)
# ==============================================================================

import re, requests, tempfile, xml.etree.ElementTree as ET
from bs4 import BeautifulSoup
import numpy as np

def _parse_prodigy_text(text: str):
    """PRODIGY HTML/텍스트에서 kcal/mol 값을 robust하게 추출"""
    pats = [
        r'Predicted\s+binding\s+affinity[^-+]*?([\-+]?\d+(?:\.\d+)?)\s*kcal\s*/\s*mol',
        r'(?:ΔG|DeltaG)\s*[:=]\s*([\-+]?\d+(?:\.\d+)?)\s*kcal\s*/\s*mol',
        r'([\-+]?\d+(?:\.\d+)?)\s*kcal\s*/\s*mol'
    ]
    for p in pats:
        m = re.search(p, text, flags=re.IGNORECASE)
        if m:
            try:
                return float(m.group(1))
            except:
                pass
    return None

def _fallback_binding_energy_from_geometry(pdb_file_path: str) -> float:
    """
    대체 결합 에너지 추정(간이): 최소 거리 + 접촉 수 기반
    더 가까울수록/접촉 많을수록 더 음수(강한 결합)로 가중
    """
    try:
        chain_a_coords, chain_b_coords = [], []
        with open(pdb_file_path, 'r') as f:
            for line in f:
                if line.startswith('ATOM'):
                    chain = line[21]
                    x, y, z = float(line[30:38]), float(line[38:46]), float(line[46:54])
                    if chain == 'A':
                        chain_a_coords.append((x, y, z))
                    elif chain == 'B':
                        chain_b_coords.append((x, y, z))

        if not chain_a_coords or not chain_b_coords:
            return 0.0

        min_distance = float('inf')
        contact_count = 0

        # 연산량 감소를 위해 Chain A는 샘플링
        a_samp = chain_a_coords[::5] if len(chain_a_coords) > 5 else chain_a_coords
        b_arr = np.array(chain_b_coords)
        for (ax, ay, az) in a_samp:
            diffs = b_arr - np.array([ax, ay, az])
            dists = np.sqrt((diffs**2).sum(axis=1))
            min_distance = min(min_distance, float(dists.min()))
            contact_count += int((dists < 5.0).sum())

        if min_distance < float('inf'):
            score = -5.0 - (10.0 / max(min_distance, 1.0)) - (contact_count * 0.1)
            return max(score, -20.0)  # 하한 캡
    except Exception as e:
        print(f"       대체 점수 계산 실패: {e}")
    return 0.0

def predict_binding_affinity_with_prodigy(pdb_file_path: str, max_retries: int = 2) -> float:
    """
    Colab 전용: PRODIGY는 '웹 모드'만 시도 → 실패 시 이 함수 내부에서 '대체 점수' 반환
    (로컬 CLI는 Colab에서 불안정하므로 사용하지 않음)
    """
    endpoints = [
        "https://rascar.science.uu.nl/prodigy/prediction",
        "https://bianca.science.uu.nl/prodigy/prediction"
    ]
    # 엔드포인트마다 폼 이름이 다를 수 있어 2종 시도
    form_variants = [
        {"file_field": "file", "data": {"temperature": "25", "chain1": "A", "chain2": "B"}},
        {"file_field": "pdbfile", "data": {"temp": "25", "chain1": "A", "chain2": "B"}},
    ]

    for ep in endpoints:
        for fv in form_variants:
            try:
                with open(pdb_file_path, "rb") as f:
                    files = {fv["file_field"]: (os.path.basename(pdb_file_path), f, "application/octet-stream")}
                    resp = requests.post(ep, files=files, data=fv["data"], timeout=60)
                if resp.status_code == 200 and resp.text:
                    txt = resp.text
                    try:
                        soup = BeautifulSoup(txt, "html.parser")
                        txt += "\n" + (soup.get_text("\n") or "")
                    except:
                        pass
                    val = _parse_prodigy_text(txt)
                    if val is not None:
                        return float(val)
            except Exception as e:
                print(f"       PRODIGY(웹:{ep}) 오류: {e}")

    # 웹 실패 시: 이 함수 내부에서 '대체 점수' 즉시 반환
    alt = _fallback_binding_energy_from_geometry(pdb_file_path)
    if alt != 0.0:
        print(f"       → PRODIGY 대체 점수 사용: {alt:.3f} kcal/mol")
    return alt

def _calculate_interactions_with_plip(pdb_file: str):
    """
    PLIP가 설치되어 있으면 PLIP로 상호작용 계산
    실패/미설치면 상위에서 대체 계산 사용
    """
    if not PLIP_AVAILABLE:
        raise RuntimeError("PLIP not available")

    with tempfile.TemporaryDirectory() as tmpd:
        cmd = ["plipcmd", "-f", pdb_file, "-o", tmpd, "-x"]
        # stderr를 숨기고 timeout 설정
        import subprocess
        subprocess.check_output(cmd, stderr=subprocess.STDOUT, universal_newlines=True, timeout=120)

        report_xml = os.path.join(tmpd, "report.xml")
        if not os.path.exists(report_xml):
            raise RuntimeError("PLIP report.xml not found")

        tree = ET.parse(report_xml)
        root = tree.getroot()

        def count_any(root, tag_substr):
            return sum(1 for el in root.iter() if tag_substr in el.tag.lower())

        h_bonds = count_any(root, "hydrogen_bond")
        hydrophobic = count_any(root, "hydrophobic_interaction")
        # salt_bridge를 정전기 상호작용으로 취급
        electrostatic = count_any(root, "salt_bridge")

        return {
            "h_bonds": h_bonds,
            "hydrophobic": hydrophobic,
            "electrostatic": electrostatic,
            "total": h_bonds + hydrophobic + electrostatic
        }

def calculate_interactions_advanced(pdb_file):
    """상호작용 계산: PLIP 우선 → 실패/미설치 시 대체(간이)"""
    # 1) PLIP 시도
    try:
        return _calculate_interactions_with_plip(pdb_file)
    except Exception as e:
        if PLIP_AVAILABLE:
            print(f"    -> PLIP 실패, 대체 계산 사용: {e}")
        else:
            print("    -> PLIP 미설치/미사용, 대체 계산 사용")

    # 2) 대체(기존 간이 계산)
    try:
        chain_a_atoms, chain_b_atoms = [], []
        with open(pdb_file, 'r') as f:
            for line in f:
                if line.startswith(('ATOM', 'HETATM')):
                    chain = line[21]
                    atom_type = line[12:16].strip()
                    element = (line[76:78].strip() or atom_type[0]).upper()
                    coords = (float(line[30:38]), float(line[38:46]), float(line[46:54]), atom_type, element)
                    if chain == 'A':
                        chain_a_atoms.append(coords)
                    elif chain == 'B':
                        chain_b_atoms.append(coords)

        if not chain_a_atoms or not chain_b_atoms:
            return {'h_bonds': 0, 'hydrophobic': 0, 'electrostatic': 0, 'total': 0}

        h_bonds = hydrophobic = electrostatic = 0
        for bx, by, bz, b_atom, b_element in chain_b_atoms:
            for ax, ay, az, a_atom, a_element in chain_a_atoms:
                distance = np.sqrt((bx-ax)**2 + (by-ay)**2 + (bz-az)**2)
                # 수소결합
                if distance <= 3.5 and ((a_element in ['N','O']) and (b_element in ['N','O'])):
                    h_bonds += 1
                # 소수성
                if distance <= 4.5 and a_element == 'C' and b_element == 'C':
                    hydrophobic += 1
                # 간이 정전기
                if distance <= 5.0:
                    charged_pos = ['LYS', 'ARG', 'HIS']
                    charged_neg = ['ASP', 'GLU']
                    a_res, b_res = a_atom[:3], b_atom[:3]
                    if ((a_res in charged_pos and b_res in charged_neg) or
                        (a_res in charged_neg and b_res in charged_pos)):
                        electrostatic += 1

        return {
            'h_bonds': h_bonds,
            'hydrophobic': hydrophobic,
            'electrostatic': electrostatic,
            'total': h_bonds + hydrophobic + electrostatic
        }
    except:
        return {'h_bonds': 0, 'hydrophobic': 0, 'electrostatic': 0, 'total': 0}



# ==============================================================================
# STEP 6: 결합력 평가 및 랭킹 (디버깅 강화 버전)
# ==============================================================================

print("\nSTEP 6: 결합력 평가 및 최종 랭킹")
print("="*80)

results = []

if predicted_pdb_files:
    for idx, pred_pdb in enumerate(predicted_pdb_files):
        print(f"\n평가 중 ({idx+1}/{len(predicted_pdb_files)}): {os.path.basename(pred_pdb)}")

        # PDB 파일 유효성 검사
        if not os.path.exists(pred_pdb):
            print("    -> PDB 파일 없음")
            continue

        file_size = os.path.getsize(pred_pdb)
        if file_size == 0:
            print("    -> 빈 PDB 파일")
            continue

        print(f"    -> PDB 파일 크기: {file_size} bytes")

        # 펩타이드 서열 매핑
        try:
            peptide_index = int(re.search(r'complex_(\d+)', os.path.basename(pred_pdb)).group(1))
            peptide_seq = peptides[peptide_index]
            print(f"    -> 펩타이드: {peptide_seq}")
        except:
            print("    -> 펩타이드 매핑 실패")
            continue

        # PDB 파일 내용 간단 확인
        try:
            with open(pred_pdb, 'r') as f:
                lines = f.readlines()
            atom_lines = [line for line in lines if line.startswith('ATOM')]
            chain_a_atoms = [line for line in atom_lines if line[21] == 'A']
            chain_b_atoms = [line for line in atom_lines if line[21] == 'B']

            print(f"    -> Chain A 원자수: {len(chain_a_atoms)}")
            print(f"    -> Chain B 원자수: {len(chain_b_atoms)}")

            if len(chain_a_atoms) == 0 or len(chain_b_atoms) == 0:
                print("    -> 경고: Chain A 또는 B가 비어있음")
        except Exception as e:
            print(f"    -> PDB 파일 읽기 오류: {e}")

        print("    -> PRODIGY 분석 시작...")
        prodigy_score = predict_binding_affinity_with_prodigy(pred_pdb)

        if prodigy_score != 0.0:
            print(f"    -> PRODIGY 결과: {prodigy_score:.3f} kcal/mol")
        else:
            print("    -> PRODIGY 결과: 0.000 (웹/대체 모두 실패)")

            # 간단한 대체 점수 (상호작용 기반)
            try:
                with open(pred_pdb, 'r') as f:
                    lines = f.readlines()

                # 최소 거리 기반 간단한 결합 에너지 추정
                chain_a_coords = []
                chain_b_coords = []

                for line in lines:
                    if line.startswith('ATOM'):
                        chain = line[21]
                        x, y, z = float(line[30:38]), float(line[38:46]), float(line[46:54])
                        if chain == 'A':
                            chain_a_coords.append((x, y, z))
                        elif chain == 'B':
                            chain_b_coords.append((x, y, z))

                if chain_a_coords and chain_b_coords:
                    min_distance = float('inf')
                    contact_count = 0

                    # 샘플링으로 계산량 감소
                    for i, (ax, ay, az) in enumerate(chain_a_coords[::5]):  # 5개마다 샘플링
                        for j, (bx, by, bz) in enumerate(chain_b_coords):
                            dist = np.sqrt((ax-bx)**2 + (ay-by)**2 + (az-bz)**2)
                            min_distance = min(min_distance, dist)
                            if dist < 5.0:  # 5Å 이내 접촉
                                contact_count += 1

                    # 거리 기반 에너지 추정
                    if min_distance < float('inf'):
                        # 가까울수록 더 음수 (안정한 결합)
                        prodigy_score = -5.0 - (10.0 / max(min_distance, 1.0)) - (contact_count * 0.1)
                        prodigy_score = max(prodigy_score, -20.0)  # 최대 -20 kcal/mol
                        print(f"    -> 대체 점수: {prodigy_score:.3f} kcal/mol (최소거리: {min_distance:.2f}Å, 접촉: {contact_count})")

            except Exception as e:
                print(f"    -> 대체 점수 계산 실패: {e}")
                prodigy_score = -5.0  # 기본값

        # 상호작용 분석
        print("    -> 상호작용 분석 중...")
        interactions = calculate_interactions_advanced(pred_pdb)
        print(f"    -> H-bonds: {interactions['h_bonds']}, Hydrophobic: {interactions['hydrophobic']}, Electrostatic: {interactions['electrostatic']}")

        # pTM 점수 확인
        ptm_score = ptm_scores_map.get(peptide_seq, 0.0)
        print(f"    -> pTM 점수: {ptm_score}")

        # 최종 점수 계산 (개선된 가중치)
        final_score = (
            abs(prodigy_score) * 0.5 +           # PRODIGY 점수 (절대값)
            interactions['total'] * 0.3 +        # 상호작용 수
            ptm_score * 15 * 0.2                 # pTM 점수 강화
        )

        results.append({
            "Peptide_Sequence": peptide_seq,
            "Final_Score": round(final_score, 3),
            "pTM_Score": ptm_score,
            "PRODIGY_Score": round(prodigy_score, 3),
            "H_bonds": interactions['h_bonds'],
            "Hydrophobic": interactions['hydrophobic'],
            "Electrostatic": interactions['electrostatic'],
            "Total_Interactions": interactions['total'],
            "Source_PDB": os.path.basename(pred_pdb)
        })

        print(f"    -> 최종 점수: {final_score:.3f}")

else:
    print("❌ 평가할 PDB 파일이 없습니다.")

print(f"\n✅ {len(results)}개 구조 평가 완료")
print("="*80)

# ==============================================================================
# STEP 7: 최종 결과 저장
# ==============================================================================

print("\nSTEP 7: 최종 결과 정리 및 저장")
print("="*80)

if results:
    import pandas as pd

    # DataFrame 생성 및 정렬
    df = pd.DataFrame(results)
    df_sorted = df.sort_values("Final_Score", ascending=False).reset_index(drop=True)

    # 엑셀 저장
    df_sorted.to_excel(OUTPUT_FINAL_XLSX_PATH, index=False)

    print("🏆 최종 펩타이드 랭킹:")
    print("=" * 100)
    print(f"{'순위':<4} {'펩타이드':<12} {'종합점수':<8} {'pTM':<6} {'PRODIGY':<8} {'상호작용':<8}")
    print("=" * 100)

    for i, row in df_sorted.head(10).iterrows():
        print(f"{i+1:<4} {row['Peptide_Sequence']:<12} {row['Final_Score']:<8.3f} "
              f"{row['pTM_Score']:<6.3f} {row['PRODIGY_Score']:<8.3f} {row['Total_Interactions']:<8}")

    print(f"\n💾 전체 결과 저장: {OUTPUT_FINAL_XLSX_PATH}")
else:
    print("❌ 평가 결과가 없습니다.")

# 실행 시간 요약
pipeline_end_time = time.time()
total_duration = pipeline_end_time - pipeline_start_time

hours = int(total_duration // 3600)
minutes = int((total_duration % 3600) // 60)
seconds = int(total_duration % 60)

# 👉 조건에 따라 출력 형식 달리하기
if hours > 0:
    print(f"\n⏱️  총 실행 시간: {hours}시간 {minutes}분 {seconds}초")
else:
    print(f"\n⏱️  총 실행 시간: {minutes}분 {seconds}초")

print("="*80)
print("🎉 펩타이드 발굴 파이프라인 완료!")
print("="*80)

STEP 0: 환경 설정 및 라이브러리 설치 (중복 방지)
🔧 첫 실행입니다. 필요한 라이브러리들을 설치합니다...

[1/6] 기본 라이브러리 설치...

[2/6] ColabFold 설치...
   > ColabFold 패치 완료

[3/6] Transformers 설치...

[4/6] 화학 정보학 도구 설치...

[5/6] AutoDock Vina 설치...

[6/6] 추가 도구 설치...

✅ 모든 라이브러리 설치 완료!

# 설치된 도구들의 상태 요약
   • AutoDock Vina: ⚠️ 간단한 추정 사용
   • PLIP: ⚠️ 대체 계산 사용
   • PRODIGY(Web): 🌐 웹 시도/실패 시 대체 점수

STEP 1: 변수 설정 및 폴더 구조 생성
✅ 프로젝트 폴더 구조 생성 완료: PDP_20250915_220801
📊 설정 요약:
   • 작업 폴더: PDP_20250915_220801
   • 펩타이드 개수: 10
   • 펩타이드 길이: 4
   • 타겟 서열 길이: 222

STEP 2: ESM-2 펩타이드 생성
ESM-2 모델 로딩: facebook/esm2_t12_35M_UR50D


tokenizer_config.json:   0%|          | 0.00/95.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/93.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/778 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/136M [00:00<?, ?B/s]


🧬 펩타이드 생성 중...
  [ 1/10] GVWD
  [ 2/10] TTGF
  [ 3/10] PSSP
  [ 4/10] KLYH
  [ 5/10] SKRS
  [ 6/10] FEHK
  [ 7/10] SWMK
  [ 8/10] DPLT
  [ 9/10] PNHG
  [10/10] SDCP

✅ 10개 펩타이드 생성 완료

STEP 3: ColabFold 3D 구조 예측
ColabFold 실행 중... (예상 시간: 10-30분)
  ✅ 복합체 0: complex_0_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000.pdb
  ✅ 복합체 1: complex_1_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000.pdb
  ✅ 복합체 2: complex_2_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000.pdb
  ✅ 복합체 3: complex_3_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000.pdb
  ✅ 복합체 4: complex_4_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000.pdb
  ✅ 복합체 5: complex_5_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000.pdb
  ✅ 복합체 6: complex_6_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000.pdb
  ✅ 복합체 7: complex_7_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000.pdb
  ✅ 복합체 8: complex_8_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000.pd